In [1]:
import pickle

def get_data(path):
    with open(path, 'rb') as f:
        data = pickle.load(f, encoding='latin1')
    return data

In [1]:
import os
filenames = []
for filename in os.listdir("RED_S9"):
    if filename.startswith('0') and filename.endswith(".pickle"):
        filenames.append(filename)

In [2]:
filenames

['0902_6_E4.pickle',
 '0913_4_E4.pickle',
 '0905_3_E4.pickle',
 '0914_7_E4.pickle',
 '0903_4_E4.pickle',
 '0915_4_E4.pickle',
 '0911_6_E4.pickle',
 '0901_6_E4.pickle',
 '0909_2_E4.pickle',
 '0906_6_E4.pickle',
 '0904_3_E4.pickle',
 '0907_4_E4.pickle',
 '0908_2_E4.pickle',
 '0910_6_E4.pickle',
 '0912_5_E4.pickle']

In [3]:
import numpy as np
def to_WESAD(data, name):
    
    d={}
    d['signal'] = {}
    d['signal']['wrist'] = {}
    d['signal']['wrist']['ACC']= data['Acceleration']
    d['signal']['wrist']['EDA']=data['EDA']
    d['signal']['wrist']['TEMP']=data['TEMP']
    d['signal']['wrist']['BVP'] = data['BVP']
    parts = name.split('_')
    if int(parts[1]) >= 4:
        d['label'] = np.ones(int(len(data['EDA'])/4) * 700, dtype=int)*2
    else:
        d['label'] = np.ones(int(len(data['EDA'])/4) * 700, dtype=int)
    return d

In [4]:
import numpy as np
def sliding_window(signal, win_len, shift):

    n_windows = int((signal.shape[0] - win_len) / shift) + 1
    samples = []
    
    for i in range(n_windows):
        samples.append(signal[i * shift: i * shift + win_len])
    
    samples = np.stack(samples)
    
    return samples

In [5]:
from sklearn.preprocessing import MinMaxScaler
def minmaxscaling(array):
    scaler = MinMaxScaler()
    scaler.fit(array)
    result = scaler.transform(array)
    return result

In [10]:
import pickle
from sklearn.preprocessing import MinMaxScaler
import os
fs = {'EDA': 4, 'TEMP': 4, 'BVP': 64, 'ACC': 32, 'label': 700}

def make_patient_data(subject_id):
    '''
    name = f'S{subject_id}'
    main_path = 'WESAD'
    with open(os.path.join(main_path, name) + '/' + name + '.pkl', 'rb') as file:
        data = pickle.load(file, encoding='latin1')
    '''
    
    with open('RED_S9/{}'.format(subject_id), 'rb') as f:
        data = to_WESAD(pickle.load(f), subject_id)
        
    WIN_LEN = 60
    SHIFT_LEN = 30
    chosen_idx = np.logical_and(data['label'] >= 1, data['label'] <= 3).sum()

    ACCx = sliding_window(minmaxscaling(data['signal']['wrist']['ACC'][:, 0:1]), fs['ACC'] * WIN_LEN, fs['ACC'] * SHIFT_LEN)
    ACCy = sliding_window(minmaxscaling(data['signal']['wrist']['ACC'][:, 1:2]), fs['ACC'] * WIN_LEN, fs['ACC'] * SHIFT_LEN)
    ACCz = sliding_window(minmaxscaling(data['signal']['wrist']['ACC'][:, 2:3]), fs['ACC'] * WIN_LEN, fs['ACC'] * SHIFT_LEN)
    EDA = sliding_window(minmaxscaling(data['signal']['wrist']['EDA']), fs['EDA'] * WIN_LEN, fs['EDA'] * SHIFT_LEN)
    TEMP = sliding_window(minmaxscaling(data['signal']['wrist']['TEMP']), fs['TEMP'] * WIN_LEN, fs['TEMP'] * SHIFT_LEN)
    BVP = sliding_window(minmaxscaling(data['signal']['wrist']['BVP']), fs['BVP'] * WIN_LEN, fs['BVP'] * SHIFT_LEN)
    windows = sliding_window(data['label'], 700 * 60, 700 * 30)
    label = np.array([np.argmax(np.bincount(win)) for win in windows])
    chosen_mask = np.logical_and(label >=1, label <= 3)
    label = (label == 2).astype(int)
    
    return ACCx[chosen_mask[:len(ACCx)]], ACCy[chosen_mask[:len(ACCy)]], ACCz[chosen_mask[:len(ACCz)]], EDA[chosen_mask[:len(EDA)]], TEMP[chosen_mask[:len(TEMP)]], BVP[chosen_mask[:len(BVP)]], label[chosen_mask[:len(label)]]


In [11]:
def make_dataset(subject_ids):
    ACCxs = []
    ACCys = []
    ACCzs = []
    EDAs = []
    TEMPs = []
    BVPs = []
    Ys = []
    
    for subject_id in subject_ids:
        ACCx, ACCy, ACCz, EDA, TEMP, BVP, label = make_patient_data(subject_id)
        ACCxs.append(ACCx)
        ACCys.append(ACCy)
        ACCzs.append(ACCz)
        EDAs.append(EDA)
        TEMPs.append(TEMP)
        BVPs.append(BVP)
        Ys.append(label)
    
    ACCxs = np.concatenate(ACCxs)
    ACCys = np.concatenate(ACCys)
    ACCzs = np.concatenate(ACCzs)
    EDAs = np.concatenate(EDAs)
    TEMPs = np.concatenate(TEMPs)
    BVPs = np.concatenate(BVPs)
    Ys = np.concatenate(Ys)
    
    return ACCxs, ACCys, ACCzs, EDAs, TEMPs, BVPs, Ys

In [12]:
print(filenames)

['0902_6_E4.pickle', '0913_4_E4.pickle', '0905_3_E4.pickle', '0914_7_E4.pickle', '0903_4_E4.pickle', '0915_4_E4.pickle', '0911_6_E4.pickle', '0901_6_E4.pickle', '0909_2_E4.pickle', '0906_6_E4.pickle', '0904_3_E4.pickle', '0907_4_E4.pickle', '0908_2_E4.pickle', '0910_6_E4.pickle', '0912_5_E4.pickle']


In [13]:
for filename in filenames:
    ACCxs, ACCys, ACCzs, EDAs, TEMPs, BVPs, Ys = make_dataset([filename])
    trainset= {}
    trainset['ACCx'] = ACCxs
    trainset['ACCy'] = ACCys
    trainset['ACCz'] = ACCzs
    trainset['EDA'] = EDAs
    trainset['TEMP'] = TEMPs
    trainset['BVP'] = BVPs
    trainset['Ys'] = Ys
    with open('RED_S9_processed/{}'.format(filename), 'wb') as handle:
        pickle.dump(trainset, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
trainset= {}
trainset['ACCx'] = ACCxs
trainset['ACCy'] = ACCys
trainset['ACCz'] = ACCzs
trainset['EDA'] = EDAs
trainset['TEMP'] = TEMPs
trainset['BVP'] = BVPs
trainset['Ys'] = Ys

In [23]:
len(Ys)

22852

In [24]:
Ys.sum()

8116

In [22]:
train[6].sum()

NameError: name 'train' is not defined

In [25]:
with open('s12.pkl', 'wb') as handle:
    pickle.dump(trainset, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [56]:
train_ids = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14]
ACCxs, ACCys, ACCzs, EDAs, TEMPs, BVPs, Ys = make_dataset(train_ids)

In [60]:
trainset= {}
trainset['ACCx'] = ACCxs
trainset['ACCy'] = ACCys
trainset['ACCz'] = ACCzs
trainset['EDA'] = EDAs
trainset['TEMP'] = TEMPs
trainset['BVP'] = BVPs
trainset['Ys'] = Ys

In [61]:
with open('train.pkl', 'wb') as handle:
    pickle.dump(trainset, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [62]:
test_ids = [15, 16, 17]
ACCxs, ACCys, ACCzs, EDAs, TEMPs, BVPs, Ys = make_dataset(test_ids)

In [63]:
testset= {}
testset['ACCx'] = ACCxs
testset['ACCy'] = ACCys
testset['ACCz'] = ACCzs
testset['EDA'] = EDAs
testset['TEMP'] = TEMPs
testset['BVP'] = BVPs
testset['Ys'] = Ys

In [64]:
trainset['ACCx'][0].shape

(1920, 1)

In [65]:
with open('test.pkl', 'wb') as handle:
    pickle.dump(testset, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [67]:
from torch.utils.data import Dataset, DataLoader
import torch

class WESADDataset(Dataset):

    def __init__(self):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.ACCx = torch.tensor(trainset['ACCx'])
        self.ACCy = torch.tensor(trainset['ACCy'])
        self.ACCz = torch.tensor(trainset['ACCz'])
        self.EDA = torch.tensor(trainset['EDA'])
        self.TEMP = torch.tensor(trainset['TEMP'])
        self.BVP = torch.tensor(trainset['BVP'])
        self.Ys = torch.tensor(trainset['Ys'])
        
        
    def __len__(self):
        return len(self.ACCx)

    def __getitem__(self, idx):
        return self.ACCx[idx], self.ACCy[idx], self.ACCz[idx], self.EDA[idx], self.TEMP[idx], self.BVP[idx], self.Ys[idx]

In [68]:
dataset = WESADDataset()

In [80]:
len(dataset)

2318

In [85]:
dataloader = DataLoader(dataset, batch_size=8,
                        shuffle=True, num_workers=0)

In [86]:
for data in dataloader:
    print(data)
    break

[tensor([[[0.3812],
         [0.3861],
         [0.3911],
         ...,
         [0.3267],
         [0.3317],
         [0.3218]],

        [[0.4565],
         [0.5000],
         [0.4957],
         ...,
         [0.3130],
         [0.3000],
         [0.3261]],

        [[0.5561],
         [0.5561],
         [0.5561],
         ...,
         [0.6537],
         [0.6439],
         [0.6439]],

        ...,

        [[0.5991],
         [0.6036],
         [0.6036],
         ...,
         [0.3333],
         [0.3333],
         [0.3288]],

        [[0.5647],
         [0.5647],
         [0.5647],
         ...,
         [0.5961],
         [0.6157],
         [0.6157]],

        [[0.3522],
         [0.3348],
         [0.3565],
         ...,
         [0.2913],
         [0.2913],
         [0.2826]]], dtype=torch.float64), tensor([[[0.4027],
         [0.4027],
         [0.4118],
         ...,
         [0.3756],
         [0.3756],
         [0.3756]],

        [[0.4863],
         [0.4824],
         [0.486

In [98]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvLayers(nn.Module):
    def __init__(self):
        super(ConvLayers, self).__init__()
        self.conv1 = nn.Conv1d(1, 128, 8, padding='same')
        self.bn1 = nn.BatchNorm1d(128)
        self.conv2 = nn.Conv1d(128, 256, 5, padding='same')
        self.bn2 = nn.BatchNorm1d(256)
        self.conv3 = nn.Conv1d(256, 128, 3, padding='same')
        self.bn3 = nn.BatchNorm1d(128)
        self.pool = nn.AdaptiveAvgPool1d(1)
        
    def forward(self, x):
        
        x = x.float()
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        return self.pool(x).squeeze(-1)
    
class FCN_model(nn.Module):
    def __init__(self, input_channels):
        super(FCN_model, self).__init__()
        # Needs code here
        self.num_channel = input_channels
        self.convLayersList = []
        for i in range(input_channels):
            self.convLayersList.append(ConvLayers())
        self.fc = nn.Linear(128 * input_channels, 2)

    def forward(self, accx, accy, accz, eda, temp, bvp):
        # Needs code here
        accx_out = self.convLayersList[0](accx)
        accy_out = self.convLayersList[1](accy)
        accz_out = self.convLayersList[2](accz)
        eda_out = self.convLayersList[3](eda)
        temp_out = self.convLayersList[4](temp)
        bvp_out = self.convLayersList[5](bvp)
        multi_modal_out = torch.cat([accx_out, accy_out,
                                     accz_out, eda_out,
                                     temp_out, bvp_out], dim=-1)
        
        return self.fc(multi_modal_out)


In [55]:
model = FCN(6)
accx = torch.zeros((30, 1, 1920))
accy = torch.zeros((30, 1, 1920))
accz = torch.zeros((30, 1, 1920))
eda = torch.zeros((30, 1, 240))
temp = torch.zeros((30, 1, 240))
bvp = torch.zeros((30, 1, 3840))

print(model(accx, accy, accz, eda, temp, bvp).shape)

torch.Size([30, 2])


In [91]:
def train_FCN(train_loader, lr=1e-3, num_epochs=10):
    '''
    Train a MLP 
    Input: X[i] is the ith training data. y[i] is the ith training label. alpha is the count prior
    Output: return a trained MLP
    '''
    # needs code here
    input_size = 784
    hidden_size = 64
    output_size = 10

    # Instantiate the model
    mlp = MLP(input_size, hidden_size, output_size)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(mlp.parameters(), lr=lr)  
    training_losses = []
    validation_losses = []

    # Train the model
    for epoch in tqdm(range(num_epochs)):
    losses = []
    for accx, accy, accz, eda, temp, bvp, label in train_loader:
        optimizer.zero_grad()
        pred = mlp(data)
        loss = loss_fn(pred, labels)
        loss.backward()
        optimizer.step()
        losses.append(loss.detach())
    training_losses.append(np.mean(losses))
    
    '''
    losses = []
    
    with torch.no_grad():
        for data, labels in val_loader:
            pred = mlp(data)
            loss = loss_fn(pred, labels)
            losses.append(loss.detach())
        validation_losses.append(np.mean(losses))

    # Display Loss Curves
    #display_error_curves(num_epochs, training_losses, validation_losses)
    '''

    return mlp, training_losses, validation_losses


IndentationError: expected an indented block (<ipython-input-91-5f31ab279bb2>, line 22)

In [102]:
from tqdm import tqdm

model = FCN_model(6)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
train_loader = DataLoader(dataset, batch_size=8,
                        shuffle=True, num_workers=0)

# Train the model
for epoch in tqdm(range(10)):
    losses = []
    for accx, accy, accz, eda, temp, bvp, label in tqdm(train_loader):
        accx = accx.permute(0, 2, 1)
        accy = accy.permute(0, 2, 1)
        accz = accz.permute(0, 2, 1)
        eda = eda.permute(0, 2, 1)
        temp = temp.permute(0, 2, 1)
        bvp = bvp.permute(0, 2, 1)
        
        optimizer.zero_grad()
        pred = model(accx, accy, accz, eda, temp, bvp)
        loss = loss_fn(pred, label)
        print(loss)
        loss.backward()
        optimizer.step()
        losses.append(loss.detach())
    print("epoch {} have loss {}".format(epoch, losses.mean()))
    
    with torch.no_grad():
        total = 0
        correct = 0
        for accx, accy, accz, eda, temp, bvp, label in tqdm(train_loader):
            accx = accx.permute(0, 2, 1)
            accy = accy.permute(0, 2, 1)
            accz = accz.permute(0, 2, 1)
            eda = eda.permute(0, 2, 1)
            temp = temp.permute(0, 2, 1)
            bvp = bvp.permute(0, 2, 1)

            pred = model(accx, accy, accz, eda, temp, bvp)
            logits = pred.argmax(-1)
            correct = correct + (labels == logits).sum().detach().cpu().numpy()
            total = total + pred.shape[0]
        print('train total:', total, 'train correct:', correct)
        print('train accuracy:', correct / total, 'train error', 1 - correct / total)

  0%|          | 0/290 [00:00<?, ?it/s]

tensor(0.6057, grad_fn=<NllLossBackward0>)



  0%|          | 1/290 [00:01<08:48,  1.83s/it]

tensor(0.4874, grad_fn=<NllLossBackward0>)



  1%|          | 2/290 [00:03<08:53,  1.85s/it]

...

tensor(0.1534, grad_fn=<NllLossBackward0>)



100%|█████████▉| 289/290 [08:45<00:01,  1.82s/it]

tensor(0.5216, grad_fn=<NllLossBackward0>)



  0%|          | 0/10 [08:47<?, ?it/s]


AttributeError: 'list' object has no attribute 'mean'